<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict outdoor equipment purchase with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon" width="800"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Scala is helpful. This notebook uses Scala 2.11 and Apache® Spark 2.1.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.
 
## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Deploy and score in a Cloud](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.bluemix.net/catalog/services/machine-learning) instance (free plan is offered).
-  Make sure that you are using a Spark 2.1 kernel.
-  Download **GoSales Transactions** from Watson Studio Community (code provided below).

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then use spark *read* method to read data. 

In [1]:
import scala.sys.process._

val filename = "GoSalesNaiveBayes.csv"

s"wget -O $filename https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600".!

--2018-04-03 07:20:15--  https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600
Resolving apsportal.ibm.com (apsportal.ibm.com)... 158.85.156.20
Connecting to apsportal.ibm.com (apsportal.ibm.com)|158.85.156.20|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dataplatform.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600 [following]
--2018-04-03 07:20:15--  https://dataplatform.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600
Resolving dataplatform.ibm.com (dataplatform.ibm.com)... 75.126.81.68
Connecting to dataplatform.ibm.com (dataplatform.ibm.com)|75.126.81.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘GoSalesNaiveBayes.csv’

     0K .......... ........

0

In [2]:
val df_data = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    option("inferSchema", "true").
    load(filename)

Waiting for a Spark session to start...

[Stage 2:=============================>                             (1 + 1) / 2]

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [3]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [4]:
df_data.show(10)

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
+--------------------+------+---+--------------+------------+
only showing top 10 rows



In [5]:
println("Total number of records: " + df_data.count())

Total number of records: 60252


As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [6]:
val splits = df_data.randomSplit(Array(0.8, 0.18, 0.02), seed = 24L)
val training_data = splits(0).cache()
val test_data = splits(1)
val prediction_data = splits(2)

println("Number of training records: " + training_data.count())
println("Number of testing records: " + test_data.count())
println("Number of prediction records: " + prediction_data.count())

Number of training records: 48176
Number of testing records: 10860
Number of prediction records: 1216


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [7]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer, IndexToString, VectorAssembler}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.{Model, Pipeline, PipelineStage, PipelineModel}
import org.apache.spark.sql.SparkSession

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [8]:
val stringIndexer_label = new StringIndexer().setInputCol("PRODUCT_LINE").setOutputCol("label").fit(df_data)
val stringIndexer_prof = new StringIndexer().setInputCol("PROFESSION").setOutputCol("PROFESSION_IX")
val stringIndexer_gend = new StringIndexer().setInputCol("GENDER").setOutputCol("GENDER_IX")
val stringIndexer_mar = new StringIndexer().setInputCol("MARITAL_STATUS").setOutputCol("MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [9]:
val vectorAssembler_features = new VectorAssembler().setInputCols(Array("GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX")).setOutputCol("features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [10]:
val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)

Finally, indexed labels back to original labels.

In [11]:
val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [12]:
val pipeline_rf = new Pipeline().setStages(Array(stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter))

Now, you can train your Random Forest model by using the previously defined **pipeline** and **training data**.

In [13]:
training_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [14]:
val model_rf = pipeline_rf.fit(training_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [15]:
val predictions = model_rf.transform(test_data)
val evaluatorRF = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val accuracy = evaluatorRF.evaluate(predictions)

println(f"Accuracy = $accuracy%.2f")

Accuracy = 0.58


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using Scala client libraries.

First, you must import client libraries.

**Note**: Apache® Spark 2.1 or higher is required.

In [16]:
// WML client library

import com.ibm.analytics.ngp.repository._

// Helper libraries

import scalaj.http.{Http, HttpOptions}
import scala.util.{Success, Failure}
import java.util.Base64
import java.nio.charset.StandardCharsets
import play.api.libs.json._

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [ ]:
val wml_credentials = """{
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}"""

In [46]:
val wml_credentials_json:JsValue = Json.parse(wml_credentials)

val wml_service_path = (wml_credentials_json \ "url").asOpt[String] match {
    case Some(x) => x
    case None => ""
}
val wml_instance_id = (wml_credentials_json \ "instance_id").asOpt[String] match {
    case Some(x) => x
    case None => ""
}
val wml_username = (wml_credentials_json \ "username").asOpt[String] match {
    case Some(x) => x
    case None => ""
}
val wml_password = (wml_credentials_json \ "password").asOpt[String] match {
    case Some(x) => x
    case None => ""
}

In [47]:
val client = MLRepositoryClient(wml_service_path)
client.authorize(wml_username, wml_password)

Success(())

Create model artifact (abstraction layer).

In [48]:
val model_artifact = MLRepositoryArtifact(model_rf, training_data, "WML Product Line Prediction Model")

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [49]:
val saved_model = client.models.save(model_artifact).get

Get saved model metadata from Watson Machine Learning.

**Tip**: Use *meta.availableProps* to get the list of available props.

In [50]:
saved_model.meta.availableProps

Vector(trainingDataSchema, lastUpdated, label, inputDataSchema, modelVersionHref, prediction, modelType, version, modelHref, artifactPath, pipelineVersionHref, runtime, creationTime, probability)

In [51]:
println("modelType: " + saved_model.meta.prop("modelType").get)
println("trainingDataSchema: " + saved_model.meta.prop("trainingDataSchema").get)
println("creationTime: " + saved_model.meta.prop("creationTime").get)
println("modelVersionHref: " + saved_model.meta.prop("modelVersionHref").get)
println("label: " + saved_model.meta.prop("label").get)

modelType: sparkml-model-2.1
trainingDataSchema: {"type":"struct","fields":[{"name":"PRODUCT_LINE","type":"string","nullable":true,"metadata":{}},{"name":"GENDER","type":"string","nullable":true,"metadata":{}},{"name":"AGE","type":"integer","nullable":true,"metadata":{}},{"name":"MARITAL_STATUS","type":"string","nullable":true,"metadata":{}},{"name":"PROFESSION","type":"string","nullable":true,"metadata":{}}]}
creationTime: 2018-04-03T12:35:32.457Z
modelVersionHref: https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/32b847bd-90a3-4db0-aa91-9fb01cf160ec/versions/605ebb83-ddde-4917-903e-4b9951c7b54c
label: PRODUCT_LINE


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load model and make predictions

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [52]:
val model_version_href = saved_model.meta.prop("modelVersionHref").get
val loaded_model_artifact = client.models.version(model_version_href).get

You can print for example model name to make sure that model artifact has been loaded correctly.

In [53]:
loaded_model_artifact.name.mkString

WML Product Line Prediction Model

As you can see the name is correct. 

In [54]:
loaded_model_artifact match {
        case SparkPipelineModelLoader(Success(model)) => {
          val predictions = model.transform(prediction_data)
        }
        case SparkPipelineModelLoader(Failure(e)) => "Loading failed."
        case _ => println(s"Unexpected artifact class: ${loaded_model_artifact.getClass}")
    }
predictions.select("GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "predictedLabel").show(10)

+------+---+--------------+-----------+--------------------+
|GENDER|AGE|MARITAL_STATUS| PROFESSION|      predictedLabel|
+------+---+--------------+-----------+--------------------+
|     F| 18|        Single|      Other|Personal Accessories|
|     F| 18|        Single|     Retail|Personal Accessories|
|     F| 19|        Single|Hospitality|   Camping Equipment|
|     F| 19|        Single|Hospitality|   Camping Equipment|
|     F| 19|        Single|Hospitality|   Camping Equipment|
|     F| 19|        Single|Hospitality|   Camping Equipment|
|     F| 19|        Single|      Other|Personal Accessories|
|     F| 19|        Single|      Other|Personal Accessories|
|     F| 19|        Single|      Other|Personal Accessories|
|     F| 19|        Single|      Other|Personal Accessories|
+------+---+--------------+-----------+--------------------+
only showing top 10 rows



By tabulating a count, you can see which product line is the most popular.

In [55]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

+--------------------+-----+
|      predictedLabel|count|
+--------------------+-----+
|   Camping Equipment| 6356|
|      Golf Equipment|  631|
|Mountaineering Eq...|  699|
|Personal Accessories| 3174|
+--------------------+-----+



You have already learned how save and load the model from Watson Machine Learning repository.

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [56]:
// Get WML service instance token

val wml_auth_header = "Basic " + Base64.getEncoder.encodeToString((wml_username + ":" + wml_password).getBytes(StandardCharsets.UTF_8))
val wml_url = wml_service_path + "/v3/identity/token"
val wml_response = Http(wml_url).header("Authorization", wml_auth_header).asString
val wml_token_json:JsValue = Json.parse(wml_response.body)

val wml_token = (wml_token_json \ "token").asOpt[String] match {
    case Some(x) => x
    case None => ""
}

### 5.1: Create online scoring endpoint

Now you can create an online scoring endpoint. Execute the following sample code that uses the publishedModelId value to create the scoring endpoint to the Bluemix repository.

#### Get published_models url from instance details

In [57]:
val endpoint_instance = wml_service_path + "/v3/wml_instances/" + wml_instance_id
val wml_response_instance = Http(endpoint_instance).header("Content-Type", "application/json").header("Authorization", "Bearer " + wml_token).option(HttpOptions.connTimeout(10000)).option(HttpOptions.readTimeout(50000)).asString

In [58]:
wml_response_instance

HttpResponse({"metadata":{"guid":"fd1db635-c9f3-4d83-afc7-32efec367e5e","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e","created_at":"2018-01-30T10:39:51.938Z","modified_at":"2018-04-03T12:35:32.514Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models"},"usage":{"expiration_date":"2018-05-01T00:00:00.000Z","computation_time":{"current":0},"model_count":{"limit":1000,"current":3},"prediction_count":{"current":0},"gpu_count":{"limit":8,"current":0},"capacity_units":{"current":0},"deployment_count":{"limit":1000,"current":2}},"plan_id":"0f2a3c2c-456b-40f3-9b19-726d2740b11c","status":"Act...

In [59]:
val published_models_json: JsValue = Json.parse(wml_response_instance.body)
val published_models_url = (((published_models_json \ "entity") \\ "published_models")(0) \ "url").as[JsString].value
published_models_url

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models

#### Get list of published models

In [60]:
val wml_models = Http(published_models_url).header("Content-Type", "application/json").header("Authorization", "Bearer " + wml_token).option(HttpOptions.connTimeout(10000)).option(HttpOptions.readTimeout(50000)).asString
wml_models

HttpResponse({"limit":1000,"resources":[{"metadata":{"guid":"32b847bd-90a3-4db0-aa91-9fb01cf160ec","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/32b847bd-90a3-4db0-aa91-9fb01cf160ec","created_at":"2018-04-03T12:35:32.457Z","modified_at":"2018-04-03T12:35:32.610Z"},"entity":{"runtime_environment":"spark-2.1","learning_configuration_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/32b847bd-90a3-4db0-aa91-9fb01cf160ec/learning_configuration","author":{},"name":"WML Product Line Prediction Model","label_col":"PRODUCT_LINE","learning_iterations_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d8...

In [61]:
var deployment_endpoint: String = _
wml_models.body.split("\"").map{ s => {if ((s contains "deployments") & (s contains saved_model.uid.mkString)) {deployment_endpoint = s}}}
deployment_endpoint

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/32b847bd-90a3-4db0-aa91-9fb01cf160ec/deployments

#### Create online deployment for published model

In [62]:
val payload_name = "Online scoring"
val payload_data_online = Json.stringify(Json.toJson(Map("type" -> "online", "name" -> payload_name)))

In [63]:
val response_online = Http(deployment_endpoint).postData(payload_data_online).header("Content-Type", "application/json").header("Authorization", "Bearer " + wml_token).option(HttpOptions.connTimeout(50000)).option(HttpOptions.readTimeout(50000)).asString

In [64]:
val scoring_url_json: JsValue = Json.parse(response_online.body)
val scoring_url = (scoring_url_json \ "entity" \ "scoring_url").asOpt[String] match {
    case Some(x) => x
    case None => ""
}

In [65]:
print(scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/32b847bd-90a3-4db0-aa91-9fb01cf160ec/deployments/ff37fa26-1076-4808-aed4-094e2fb97d29/online

In [66]:
val payload_scoring = Json.stringify(Json.toJson(Map("fields" -> Json.toJson(List(Json.toJson("GENDER"), Json.toJson("AGE"), Json.toJson("MARITAL_STATUS"), Json.toJson("PROFESSION"))),
                                                    "values" -> Json.toJson(List(List(Json.toJson("M"), Json.toJson(55), Json.toJson("Single"), Json.toJson("Executive")))))))

In [67]:
payload_scoring

{"fields":["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values":[["M",55,"Single","Executive"]]}

Now, you can send (POST) new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [68]:
val response_scoring = Http(scoring_url).postData(payload_scoring).header("Content-Type", "application/json").header("Authorization", "Bearer " + wml_token).option(HttpOptions.method("POST")).option(HttpOptions.connTimeout(10000)).option(HttpOptions.readTimeout(50000)).asString

In [69]:
print(response_scoring)

HttpResponse({
  "fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "PRODUCT_LINE", "label", "PROFESSION_IX", "GENDER_IX", "MARITAL_STATUS_IX", "features", "rawPrediction", "probability", "prediction", "predictedLabel"],
  "values": [["M", 55, "Single", "Executive", "Camping Equipment", 0.0, 3.0, 0.0, 1.0, [0.0, 55.0, 1.0, 3.0], [2.5050408694752604, 1.8771700964659068, 2.338808434620882, 3.1481331680893025, 0.13084743134864898], [0.250504086947526, 0.1877170096465907, 0.2338808434620882, 0.31481331680893027, 0.013084743134864898], 3.0, "Golf Equipment"]]
},200,Map(Cache-Control -> Vector(private, no-cache, no-store, must-revalidate), Connection -> Vector(Keep-Alive), Content-Type -> Vector(application/json), Date -> Vector(Tue, 03 Apr 2018 12:36:39 GMT), Pragma -> Vector(no-cache), Server -> Vector(nginx/1.13.6), Status -> Vector(HTTP/1.1 200 OK), Transfer-Encoding -> Vector(chunked), X-Backside-Transport -> Vector(OK OK), X-Content-Type-Options -> Vector(nosniff), X-Envoy-Ups

As we can see we predict that a 55-year-old single male executive is interested in Mountaineering Equipment (prediction: 2.0).

<a id="summary"></a>
## 6. Summary and next steps 

You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 
 
### Authors

**Umit Mert Cakmak** is Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable insights.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.